In [1]:
import os 
%pwd

'/Applications/AI/GAN/research'

In [2]:
os.chdir('../')
%pwd

'/Applications/AI/GAN'

In [3]:
os.listdir()

['research',
 'requirements.txt',
 'config',
 'params.yaml',
 'README.md',
 'setup.py',
 'dvc.yaml',
 'logs',
 '.venv',
 '.github',
 'template.py',
 'templates',
 '.git',
 'main.py',
 '.idea',
 'src']

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Data Ingestion: config
from src.GAN.constants import *
from src.GAN.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
# Data Ingestion: Components
import os
import zipfile
import gdown
from src.GAN import logger
from src.GAN.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> None:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self) -> None:
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

# Data Ingestion: pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

# if error check pwd or restart program

[2025-03-20 09:08:29,563: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-20 09:08:29,567: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-20 09:08:29,568: INFO: common: created directory at: artifacts]
[2025-03-20 09:08:29,571: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-20 09:08:29,572: INFO: 2177093844: Downloading data from https://drive.google.com/file/d/1OsDiZ9Kyo-Qxa1xYKVWAZPasFgzAdjcP/view?usp=sharing into file artifacts/data_ingestion/data.zip]


/Applications/AI/GAN/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1OsDiZ9Kyo-Qxa1xYKVWAZPasFgzAdjcP
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1OsDiZ9Kyo-Qxa1xYKVWAZPasFgzAdjcP&confirm=t&uuid=a47bd314-dadc-4db3-88c7-2748151f1b83
To: /Applications/AI/GAN/artifacts/data_ingestion/data.zip
100%|██████████| 1.63G/1.63G [08:05<00:00, 3.36MB/s]

[2025-03-20 09:16:42,368: INFO: 2177093844: Downloaded data from https://drive.google.com/file/d/1OsDiZ9Kyo-Qxa1xYKVWAZPasFgzAdjcP/view?usp=sharing into file artifacts/data_ingestion/data.zip]
